In [8]:
import os
import pprint
with open(".env","r") as f:
    key = f.read().strip()
os.environ["OPENAI_API_KEY"] = key

## Interacting with OpenAI programmatically 
* Set up a client
* Create a prompt
* To provide a context for the prompt, use a message object
* Specify the openAI model you can plan on using
* Make an API Call

In [5]:
# Client
import openai
from openai import OpenAI
client = OpenAI()

# Prompt
 
prompt = "Who holds the fastest 100 metres record?"

#Message
#Let's say we onlt want the record for the Olympics
#We'll use the system attribute to set the context

context = "Your answers should be confined to the summer olympics"
message =[
    {"role": "system", "content": context},
    {"role": "user", "content": prompt}
]

# Specify the model
model = "gpt-4o"

# Call the API
response = client.chat.completions.create (
    #Temperature determines how creative the response should be, if you want creative answers set temperature to more than 0
    model = model,
    messages = message,
    temperature=0
)

# Preety print the response
import pprint
pprint.pprint(response.choices[0].message.content)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
response

In [9]:
def get_completion(message, client=client, model="gpt-4o", temperature=0):
    message=message
    response = client.chat.completions.create(
        model = model,
        messages = message,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
prompt = """I have data that consists of:
* 10 independent features, all normally distributed
* one dependent variable with binary 0 or 1 values
Generate sample data (100 samples) and write python code 
using the sklearn library for 3 different prediction models. 
Report their AUC scores"""

message=[
    {"role": "system", "content":"You are a machine learning expert"},
    {"role": "user", "content": prompt}
]

response = get_completion(message=message)
print(response)

- Ask a follow up question
* Set the context by including the previous prompt and response
* Then ask the question

In [ ]:
prompt_2 = """Thank you. That was very helpful. For the same problem, could you create python code
for a simple neural network model using tensorflow and keras?"""

message = [
    {"role": "system", "content": "You are a machine learning expert"},
    {"role": "user", "content": prompt},
    {"role": "system", "content": response},
    {"role": "user", "content": prompt_2}
]
print(get_completion(message=message))

The fastest 100m example again

In [ ]:
prompt = "Who holds the fastest 100 metres record, the fastest 200m record and the 400m record?"
message = [
    {"role": "system", "content": "You are an athletics expert"},
    {"role": "user", "content": prompt}
]
pprint.pprint(get_completion(message=message))

In [ ]:
prompt_2 = "Only for the Olympics"

message = [
    {"role": "system", "content": "You are an athletics expert"},
    {"role": "user", "content": prompt},
    {"role": "system", "content": response},
    {"role": "user", "content": prompt_2}
]
pprint.pprint(get_completion(message=message))

In [ ]:
import pprint
prompt = """
url: "http://www.xe.com/currencytables/from="USD" 

Read the content from the url above
"""

message = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": prompt}
]

pprint.pprint(get_completion(message=message))

- Langchain

In [ ]:
!pip install openai==1.59.4
!pip install langchain==0.3.14
!pip install langchain_openai==0.2.14
!pip install langchain_experimental==0.3.4
!pip install langchain_community==0.3.14

- Agents
* LangChain itneracts with LLMs using an agent
* Create an OpenAI agent and invoke it to interact with chatgpt at the chat level

In [ ]:
import pprint
from langchain_openai import ChatOpenAI
agent = ChatOpenAI(temperature=0.0)
answer = agent.invoke("Is it easy to use generative AI in my work?")
pprint.pprint(answer.content)

- Analyzing Data
* Langchain has a library for working with pandas dataframes
* We'll use the dataset "subscriptions.csv" for some basic analysis
* This dataset is an extract of a dataset on kaggle

- Rough steps in using langchain and openai for data analysis
* Read the data into a pandas dataframe
* Parameterize a langchain LLM agent
* Create the agent that contains both the df as well as the LLM
* Ask the agent questions

In [ ]:
import pandas as pd
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

# STEP 1: Create a dataframe
df = pd.read_csv("subscription.csv")
# Drop Nans
df.dropna(inplace=True)

# Step 2: Initialize the OpenAI LLM
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0,
    api_key = key
)

# Step 3: Create a pandas agent
agent = create_pandas_dataframe_agent(llm,df,
                                      agent_type ="openai-tools",
                                      allow_dangerous_code=True
                                      verbose=True)

# Step 4: Interact with the agent
print("Welcome to the subscription analyzer")
query = input("Ask a question:")

response = agent.invoke(query)
import pprint
print(pprint.pformat(response))

In [ ]:
query = "How many rows in the subscription table?"
output = agent.invoke(query)
pprint.pprint(output)